<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/harry_potter/HarryPotterNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install -U selenium neo4j spacy==2.3.2 neo4j

# Setup SpaCy and NeuralCoref
from google.colab import drive
drive.mount('/content/gdrive')

%rm neuralcoref
!git clone https://github.com/huggingface/neuralcoref.git
!pip install -U 
!python -m spacy download en_core_web_sm

%cd neuralcoref

!pip install -r requirements.txt
!pip install -e .

Requirement already up-to-date: selenium in /usr/local/lib/python3.7/dist-packages (3.141.0)
Requirement already up-to-date: neo4j in /usr/local/lib/python3.7/dist-packages (4.3.3)
Requirement already up-to-date: spacy==2.3.2 in /usr/local/lib/python3.7/dist-packages (2.3.2)
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
rm: cannot remove 'neuralcoref': Is a directory
fatal: destination path 'neuralcoref' already exists and is not an empty directory.
ERROR: You must give at least one requirement to install (see "pip help install")
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
/content/neuralcoref
Obtaining file:///content/neuralcoref
  Found existing installation: neuralcoref 4.0
    Can't uninstall 'neuralcoref'. No files were found to uninstall.
  Running setup.py develop for neuralcoref


## Restart runtime in order for SpaCy and NeuralCoref to work

In [2]:
# Setup selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [3]:
def enrich_single_item(item):
  try:
    # Find the HTML element with required data
    div = wd.find_element_by_xpath(f"//div[@data-source = '{item}']")
    # Extract relevant data from "a" or "div" tag
    try:
      result = div.find_element_by_tag_name("a").text.split('[')[0].strip()
      if result[0] == '[':
        raise Exception
    except:
      result = div.find_element_by_tag_name("div").text.split('[')[0].strip()
    return result
  except:
    return None
  

In [4]:
import time

def get_characters(url):
  # Get the list of characters by chapter
  wd.get(url)
  character_dict = dict()
  elem = wd.find_element_by_class_name("mw-parser-output")
  
  # Locate character by chapter
  tables = elem.find_elements_by_tag_name('table')
  for i, chapter in enumerate(tables):
    list_of_characters = []
    characters = chapter.find_elements_by_tag_name('a')
    for character in characters:
      if not character.get_attribute('title'):
        continue
      list_of_characters.append({'title': character.get_attribute('title'), 'url': character.get_attribute('href')})
    character_dict['chapter_' + str(i + 1)] = list_of_characters
  # Enrich characters with additional information
  for chapter in character_dict:
    for index, character in enumerate(character_dict[chapter]):
      # Rate limit sleep
      time.sleep(1)
      # Get the character page with selenium
      wd.get(character['url'])
      # Enrich aliases
      try:
        alias_div = wd.find_element_by_xpath("//div[@data-source = 'alias']")
        aliases = alias_div.find_elements_by_tag_name('li')
        result = []
        for a in aliases:
          # Ignore under the cloak-guise and the name he told
          if "disguise" in a.text or "the name he told" in a.text:
            continue
          alias = a.text.split('[')[0].split('(')[0].strip()
          result.append(alias)
        character_dict[chapter][index]['aliases'] = result
      except:
        pass
      # Enrich loyalties
      try:
        loyalty_div = wd.find_element_by_xpath("//div[@data-source = 'loyalty']")
        loyalties = loyalty_div.find_elements_by_tag_name('li')
        result = []
        for l in loyalties:
          # Ignore under the cloak-guise and the name he told
          loyalty = l.text.split('[')[0].split('(')[0].strip()
          result.append(loyalty)
        character_dict[chapter][index]['loyalty'] = result
      except:
        pass
      # Enrich family relationships
      try:
        family_div = wd.find_element_by_xpath("//div[@data-source = 'family']")
        relationships = family_div.find_elements_by_tag_name('li')
        result = []
        for r in relationships:
          rel = r.text.split('[')[0].split('(')[0].strip()
          rel_type = r.text.split('(')[-1].split(')')[0].split('[')[0]
          result.append({'person':rel, 'type': rel_type})
        character_dict[chapter][index]['family'] = result
      except:
        pass
      # Enrich blood
      character_dict[chapter][index]['blood'] = enrich_single_item('blood')
      # Enrich nationality
      character_dict[chapter][index]['nationality'] = enrich_single_item('nationality')
      # Enrich species
      character_dict[chapter][index]['species'] = enrich_single_item('species')
      # Enrich house
      character_dict[chapter][index]['house'] = enrich_single_item('house')
      # Enrich gender
      character_dict[chapter][index]['gender'] = enrich_single_item('gender')
  return character_dict




    

In [5]:
character_dict = get_characters("https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Philosopher%27s_Stone_(character_index)")

In [6]:
len(character_dict)

17

In [7]:
from neo4j import GraphDatabase
# Change the host and user/password combination to your neo4j
# Will not work with a localhost bolt url
host = 'bolt://3.239.224.171:7687'
user = 'neo4j'
password = 'rainbow-mittens-track'
driver = GraphDatabase.driver(host,auth=(user, password))

In [8]:
entity_query = """
UNWIND $data as row
MERGE (c:Character{name:row.title})
SET c.url = row.url,
    c.aliases = row.aliases,
    c.blood = row.blood,
    c.nationality = row.nationality,
    c.species = row.species,
    c.gender = row.gender
FOREACH (h in CASE WHEN row.house IS NOT NULL THEN [1] ELSE [] END | MERGE (h1:House{name:row.house}) MERGE (c)-[:BELONGS_TO]->(h1))
FOREACH (l in row.loyalty | MERGE (g:Group{name:l}) MERGE (c)-[:LOYAL_TO]->(g))
FOREACH (f in row.family | MERGE (f1:Character{name:f.person}) MERGE (c)-[t:FAMILY_MEMBER]->(f1) SET t.type = f.type)    

"""
with driver.session() as session:
  for chapter in character_dict:
    session.run(entity_query, {'data': character_dict[chapter]})


In [9]:
def get_character_dict(chapter):
  super_list = list()
  dicts = [character_dict['chapter_' + str(i)] for i in range(1,chapter + 1)]
  for d in dicts:
    for item in d:
      super_list.append(item)
  return super_list

In [10]:
import spacy
from spacy.matcher import Matcher
import neuralcoref

nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

doc1 = nlp('My sister has a dog. She loves him.')
print(doc1._.coref_clusters)

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)


[My sister: [My sister, She], a dog: [a dog, him]]


In [11]:
def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

In [12]:
import random

hardcoded_options = dict()
hardcoded_options['Malfoy'] = ['Draco Malfoy']
hardcoded_options['Patil'] = ['Padma Patil', 'Parvati Patil']
hardcoded_options['Tom'] = ['Tom']



def handle_multiple_options(result, doc):
  needs_deduplication = [(i,x) for i,x in enumerate(result) if len(x['string_id']) > 1]
  for index, multiple_options in needs_deduplication:
    # Special logic for Dursleys, if there if Mr. then Vernon, if Mrs. then Petunia
    prefix = doc[multiple_options['start']-3 : multiple_options['start']]
    if (multiple_options['text'] == 'Dursley') and ("Mr." in prefix.text):
      resolution = ["Vernon Dursley"]
    elif (multiple_options['text'] == 'Dursley') and ("Mrs." in prefix.text):
      resolution = ["Petunia Dursley"]
    # Find nearest entity
    else:
      end_char = multiple_options['end']
      distance = sys.maxsize
      resolution = []
      for possible_option in result:
        # Skip multiple options and entities that don't have any of the multiple option
        if (not len(possible_option['string_id']) == 1) or (not possible_option['string_id'][0] in multiple_options['string_id']):
          continue
        new_distance = abs(multiple_options['end'] - possible_option['end'])
        if new_distance < distance:
          distance = new_distance
          resolution = possible_option['string_id']
      
      if not resolution:
        try:
          ho = hardcoded_options[multiple_options['text']]
          if len(ho) == 1:
            resolution = ho
          else:
            resolution = [random.choice(ho)]
        except:
          print(f"no way to disambiguate {multiple_options['text']} from options: {multiple_options['string_id']}")
    
    result[index]['string_id'] = resolution
  
  if [el for el in result if len(el['string_id']) > 1]:
    print("Those entities were not correctly disambiguated")
    print([el for el in result if len(el['string_id']) > 1])
  return result

In [13]:
from collections import Counter

def get_distances(result, distance_threshold):
  #sort by start character
  result = sorted(result, key=lambda k: k['start'])
  compact_entities = []
  for entity in result:
    # If the same entity occurs, prolong the end 
    if (len(compact_entities) > 0) and (compact_entities[-1]['string_id'] == entity['string_id']):
      compact_entities[-1]['end'] = entity['end']
    else:
      compact_entities.append(entity)
  distances = list()
  # We can skip the last entity
  for index, source in enumerate(compact_entities[:-1]):
    for target in compact_entities[index + 1:]:
      if (source['string_id'] != target['string_id']) and (abs(source['end'] - target['start']) < distance_threshold):
        try:
          link = sorted([source['string_id'][0], target['string_id'][0]])
        except:
          print(source['string_id'], target['string_id'])
        distances.append(link)
      else:
        break
  return Counter(map(tuple, distances))
  



In [14]:
def store_to_neo4j(distances):
  data = [{'source': el[0], 'target': el[1], 'weight': distances[el]} for el in distances]
  with driver.session() as session:
    session.run("""
    UNWIND $data as row
    MERGE (c:Character{name:row.source})
    MERGE (t:Character{name:row.target})
    MERGE (c)-[i:INTERACTS]-(t)
    SET i.weight = coalesce(i.weight,0) + row.weight
    """, {'data': data})

In [15]:
import requests

def get_text(url):
  req = requests.get(url)
  req = req.text
  return req

In [16]:
def get_matcher_patterns(character):
  matcher_pattern = []
  stop_words = ['of', 'the', 'at', 'family', 'keeper', 'wizard', 'fat', 'de', 'hogwarts']
  parts_of_name = [el for el in character['title'].split(' ') if len(el) > 2]
  # Append the whole pattern
  matcher_pattern.append([{"LOWER": n.lower(), "IS_TITLE": True} for n in parts_of_name])
  
  # Append parts of names
  if not "'" in character['title']: # Skip names like Vernon Dursley's secretary
    for n in parts_of_name:
      if n.lower() in stop_words: # Skip appending stop words
        continue
      matcher_pattern.append([{"LOWER": n.lower(), "IS_TITLE": True}])
      # Special case for Ronald Weasley -> Also add Ron
      if n == "Ronald":
        matcher_pattern.append([{"LOWER": "ron", "IS_TITLE": True}])
  return matcher_pattern

In [17]:
url = "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"
text = get_text(url)
chapters = text.split("CHAPTER")[1:]
def get_characters_in_chapter(chapter):
  c = chapters[chapter - 1]
  # Prepare characters matcher
  matcher = Matcher(nlp.vocab)
  for character in get_character_dict(chapter):
    matcher_pattern = get_matcher_patterns(character)
    matcher.add(character['title'], matcher_pattern)

  # Prepare text
  lines = c.split('\n')[1:]
  lines = list(filter(None, lines))
  chapter_title = lines[0]
  print(chapter_title)
  text = " ".join(lines[1:])
  
  # Run coreference resolution
  text = coref_resolution(text)

  # Find matches
  doc = nlp(text)
  matches = matcher(doc)
  result = []
  for match_id, start, end in matches:
      string_id = nlp.vocab.strings[match_id]  # Get string representation
      span = doc[start:end]  # The matched span

      # Get predicates for correct result appendment
      exists_longer = [(start == e['start'] and end < e['end']) or (start > e['start'] and end == e['end']) for e in result]
      same = [start == e['start'] and end == e['end'] for e in result]
      shorter_end = [start == e['start'] and end > e['end'] for e in result]
      shorter_start = [start < e['start'] and end == e['end'] for e in result]
      
      # Append to results
      if any(exists_longer): # If there is a longer version of the given entity already in results
        continue
      
      if any(shorter_end): # If there is any entity with the same start span but has shorter end
        del result[shorter_end.index(True)]
        result.append({'string_id': [string_id], 'start': start, 'end': end, 'text': span.text}) 
      elif any(shorter_start): # If there is any entity with the same end span but has shorter start
        del result[shorter_start.index(True)]
        result.append =({'string_id': [string_id], 'start': start, 'end': end, 'text': span.text}) 
      elif not any(same): # If not exists yet
        result.append({'string_id': [string_id], 'start': start, 'end': end, 'text': span.text})
      else: # Add more entities to a single span
        i = same.index(True)
        result[i]['string_id'].append(string_id)
  
  # Handle results where there are multiple options
  handle_multiple_options(result, doc)
  return result




In [18]:
for c in range(1,len(chapters) + 1):
  end = get_characters_in_chapter(c)
  distances = get_distances(end, 14)
  store_to_neo4j(distances)

THE BOY WHO LIVED
THE VANISHING GLASS
THE LETTERS FROM NO ONE
THE KEEPER OF THE KEYS
DIAGON ALLEY
THE JOURNEY FROM PLATFORM NINE AND THREE-QUARTERS
THE SORTING HAT
THE POTIONS MASTER
THE MIDNIGHT DUEL
HALLOWEEN
QUIDDITCH
THE MIRROR OF ERISED
NICOLAS FLAMEL
NORBERT THE NORWEGIAN RIDGEBACK
THE FORIBIDDEN FOREST
THROUGH THE TRAPDOOR
THE MAN WITH TWO FACES
